In [ ]:
# HTSOHM workflow 0.9.1

This notebook takes a user through the process of generating a seed population of hypothetical porous materials and successively mutating libraries of said materials until the method as converged--that is the structure-property space has been sufficiently uniformly sampled.

In [1]:
HTSOHM_dir = '~/HTSOHM-dev' #specifies HTSOHM directory
import sys
sys.path.insert(0, './bin') #adds HTSOHM modules to Python

number_of_atom_types = 4
number_of_materials = 50

bins = 5

## Generate seed population 

Now that the number of atom-types and population size have been specified, the seed can be generated.

In [ ]:
from generate import *

generate(number_of_materials,
         number_of_atom_types)

## Screen seed

Now that the seed population has been generated, each material can then be screened using RASPA. Because the number of materials is typically large, the work is distributed across a computing cluster. For each material methane loading (see `${HTSOHM}/bin/ch4simNOHVF` for RASPA input file), void fraction (see `${HTSOHM}/bin/HVsim`), and surface area (see `${HTSOHM}/bin/SAsim`) is calculated. Raw data will be saved in `${HTSOHM}/data/`.

<b>DEV NOTE: ln 17, screen.sh<br>
`export HTSOHM_DIR=${HOME}/HTSOHM-dev`<br>
`export LIB_DIR=${HOME}/HTSOHM-dev/$LIBRARY_DIR`'<br>

ALSO MATS PER CORE REDUCED FROM 100 TO 20

In [ ]:
%run ./bin/screen.ipy

screen(HTSOHM_dir, 'gen0', 0, number_of_materials)

## Collect output

Data for the entire library is now divided into different directories, one for each node used to carry out the calculations. All of this data is then collected into three files, one for each of the properties calculated: `ch4_abs_cc_cc.txt`, `HVdata2col.txt`, and `SAdata_m2_cc.txt`. These data files can be find found in `${HTSOHM}/data/` and the directory containing all materials from that library.

<b>DEV NOTE: rmdata is commented out...<br>
`find_missing` only checks names, not values

In [ ]:
%run ./bin/cat_data.ipy
from find_missing import *

prep_gen0(HTSOHM_dir, number_of_materials)
find_missing('gen0')

## Bin library, select parents

Now that structure-properties have been calculated for the seed population, the library can be binned--sorted in three dimensions: methane loading, surface area, and void fraction--so that those materials with the rarest combinations of structure properties can be selected to <i>parent</i> new materials.

In [ ]:
from binning import *

p_dir = 'gen0'
#bins = 5
n_child = number_of_materials

bin_count_gen0, bin_IDs_gen0 = bin3d('gen0', bins)

p_list_gen0 = pick_parents('gen0',
                           bin_count_gen0,
                           bin_IDs_gen0,
                           n_child)

## <i>Dummy</i> test

Because properties are calculated using MC methods, it is possible that a partiular data point (combination of properties) seems to be unique, but is in fact a statistical anomaly. To deal with this, each parent is re-screened 5 times so that the results can be compared to the original calculation.

In [ ]:
%run ./bin/dummy_screen.ipy

dummy_screen(HTSOHM_dir, 'gen0')

<b>DEV NOTE: add find missing step???

In [ ]:
%run ./bin/dummy_test.ipy

dummy_test(HTSOHM_dir, 'gen0')

# First generation 

## Mutate `gen0`, create `gen1`

In [2]:
mutation_strength = 0.2

from mutate import *

firstS('gen0', mutation_strength, bins)
mutate('gen0', number_of_atom_types, 'gen1')

making 50 materials...
...done!


## Screen `gen1`

In [3]:
%run ./bin/screen.ipy

screen(HTSOHM_dir,
       'gen1',
       1 * number_of_materials,
       number_of_materials)

/ihome/cwilmer/ark111/HTSOHM-dev/bin
Preparing to submit jobs...
See terminal for status.

Jobs submitted!
/ihome/cwilmer/ark111/HTSOHM-dev


## Collect `gen1` output

In [3]:
%run ./bin/cat_data.ipy
from find_missing import *

prep4mut(HTSOHM_dir,
         'gen1',
         1 * number_of_materials,
         number_of_materials,
         'gen0',
         'tgen1')

find_missing('tgen1')

/ihome/cwilmer/ark111/HTSOHM-dev/bin
/ihome/cwilmer/ark111/HTSOHM-dev
No missing data points!


([], [], [])

## Bin `tgen1`, select parents 

In [4]:
from binning import *

p_dir = 'tgen1'
n_child = number_of_materials

bin_count, bin_IDs = bin3d(p_dir, bins)

p_list_tgen1 = pick_parents(p_dir,
                           bin_count,
                           bin_IDs,
                           n_child)

## `tgen1` dummy test

In [3]:
%run ./bin/dummy_screen.ipy

dummy_screen(HTSOHM_dir, 'tgen1')

/ihome/cwilmer/ark111/HTSOHM-dev/bin
Screening dummies...
See terminal for status.

Jobs submitted!
/ihome/cwilmer/ark111/HTSOHM-dev


In [2]:
%run ./bin/dummy_test.ipy

dummy_test(HTSOHM_dir, 'tgen1')

/ihome/cwilmer/ark111/HTSOHM-dev/bin
/ihome/cwilmer/ark111/HTSOHM-dev


ValueError: Some errors were detected !
    Line #26 (got 1 columns instead of 1)
    Line #27 (got 1 columns instead of 1)
    Line #28 (got 1 columns instead of 1)
    Line #29 (got 1 columns instead of 1)
    Line #30 (got 1 columns instead of 1)
    Line #36 (got 1 columns instead of 1)
    Line #37 (got 1 columns instead of 1)
    Line #38 (got 1 columns instead of 1)
    Line #39 (got 1 columns instead of 1)
    Line #40 (got 1 columns instead of 1)
    Line #81 (got 1 columns instead of 1)
    Line #82 (got 1 columns instead of 1)
    Line #83 (got 1 columns instead of 1)
    Line #84 (got 1 columns instead of 1)
    Line #85 (got 1 columns instead of 1)
    Line #91 (got 1 columns instead of 1)
    Line #92 (got 1 columns instead of 1)
    Line #93 (got 1 columns instead of 1)
    Line #94 (got 1 columns instead of 1)
    Line #95 (got 1 columns instead of 1)

# Second generation

## Calculate mutation strength(s), create `gen2`

In [ ]:
from mutate import *

calc_S('tgen1', bins)
mutate('tgen1', number_of_atom_types, 'gen2')

## Screen `gen2`

In [ ]:
%run ./bin/screen.ipy

screen(HTSOHM_dir,
       'gen2',
       2 * number_of_materials,
       number_of_materials)